In [1]:
#import all the librairies
import pandas as pd
import zipfile
import kaggle

In [2]:
#download dataset form kaggle
!kaggle datasets download -d hmavrodiev/london-bike-sharing-dataset

london-bike-sharing-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
#extract the file from the download zip file
zipfile_name='london-bike-sharing-dataset.zip'
with zipfile.ZipFile(zipfile_name, 'r') as file:
    file.extractall()

In [4]:
#read the csv file
df=pd.read_csv("london_merged.csv")

In [5]:
#explore the file
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17414 entries, 0 to 17413
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   timestamp     17414 non-null  object 
 1   cnt           17414 non-null  int64  
 2   t1            17414 non-null  float64
 3   t2            17414 non-null  float64
 4   hum           17414 non-null  float64
 5   wind_speed    17414 non-null  float64
 6   weather_code  17414 non-null  float64
 7   is_holiday    17414 non-null  float64
 8   is_weekend    17414 non-null  float64
 9   season        17414 non-null  float64
dtypes: float64(8), int64(1), object(1)
memory usage: 1.3+ MB


In [6]:
df.shape

(17414, 10)

In [7]:
df

,timestamp,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0
1,2015-01-04 01:00:00,138,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0
2,2015-01-04 02:00:00,134,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0
3,2015-01-04 03:00:00,72,2.0,2.0,100.0,0.0,1.0,0.0,1.0,3.0
4,2015-01-04 04:00:00,47,2.0,0.0,93.0,6.5,1.0,0.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...
17409,2017-01-03 19:00:00,1042,5.0,1.0,81.0,19.0,3.0,0.0,0.0,3.0
17410,2017-01-03 20:00:00,541,5.0,1.0,81.0,21.0,4.0,0.0,0.0,3.0
17411,2017-01-03 21:00:00,337,5.5,1.5,78.5,24.0,4.0,0.0,0.0,3.0
17412,2017-01-03 22:00:00,224,5.5,1.5,76.0,23.0,4.0,0.0,0.0,3.0


#Metadata

#"timestamp" - timestamp field for grouping the data

"cnt" - the count of a new bike shares

"t1" - real temperature in C

"t2" - temperature in C "feels like"

"hum" - humidity in percentage

"wind_speed" - wind speed in km/h

"weather_code" - category of the weather

"is_holiday" - boolean field - 1 holiday / 0 non holiday

"is_weekend" - boolean field - 1 if the day is weekend

"season" - category field meteorological seasons: 0-spring ; 1-summer; 2-fall; 3-winter.

"weathe_code" category description:

1 = Clear ; mostly clear but have some values with haze/fog/patches of fog/ fog in vicinity 2 = scattered clouds / few 
clouds 3 = Broken clouds 4 = Cloudy 7 = Rain/ light Rain shower/ Light rain 10 = rain with thunderstorm 26 = snowfall 94 =
 Freezing Fog

Let's explore "weather_code" and "season" columns

In [8]:
#count of the unique values in the "weather_code" columns
df.weather_code.value_counts()

weather_code
1.0     6150
2.0     4034
3.0     3551
7.0     2141
4.0     1464
26.0      60
10.0      14
Name: count, dtype: int64

In [9]:
df.season.value_counts()

season
0.0    4394
1.0    4387
3.0    4330
2.0    4303
Name: count, dtype: int64

Let's change some columns label into some meaningfull names

In [10]:
#create a dictionary 
new_names_cols={'timestamp':'time','cnt':'count','t1':'temp_real_C','t2':'temp_feels_like_C','hum':'humidity_percent','wind_speed':'wind_speed','weather_code':'weather','is_holiday':'holiday','is_weekend':'weekend','season':'season'}
#renaming columns with new dictionary
df.rename(new_names_cols,axis=1,inplace=True)

In [11]:
#Taking a look to the new dataframe 
df.head()

,time,count,temp_real_C,temp_feels_like_C,humidity_percent,wind_speed,weather,holiday,weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0
1,2015-01-04 01:00:00,138,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0
2,2015-01-04 02:00:00,134,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0
3,2015-01-04 03:00:00,72,2.0,2.0,100.0,0.0,1.0,0.0,1.0,3.0
4,2015-01-04 04:00:00,47,2.0,0.0,93.0,6.5,1.0,0.0,1.0,3.0


Let's now transform some of the datas

In [12]:
#Let's tranform humidity into percent(0 to 1)
df.humidity_percent=df.humidity_percent/100

#Let's transform weather's observations into meaningful strings
weather_dict={1.0:'clear', 2.0:'scattered_clouds',3.0:'Broken_clouds',4.0:'Cloudy',7.0:'Rain',10.0:'rain with thdstrm',26.0:'snowfall',94.0:'Freezing fog'}
df.weather=df.weather.map(weather_dict)

#Let's transform season's observations into string
season_dic={0.0:'spring',1.0:'summer',2:'fall',3:'winter'}
df.season=df.season.map(season_dic)

#Let's transform holiday and weekend's observations into booleans
holiday_dic={0.0:'No',1.0:'yes'}
df.holiday=df.holiday.map(holiday_dic)
weekend_dic={0.0:'No',1.0:'yes'}
df.weekend=df.weekend.map(weekend_dic)

In [13]:
df.head()

,time,count,temp_real_C,temp_feels_like_C,humidity_percent,wind_speed,weather,holiday,weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,0.930,6.0,Broken_clouds,No,yes,winter
1,2015-01-04 01:00:00,138,3.0,2.5,0.930,5.0,clear,No,yes,winter
2,2015-01-04 02:00:00,134,2.5,2.5,0.965,0.0,clear,No,yes,winter
3,2015-01-04 03:00:00,72,2.0,2.0,1.000,0.0,clear,No,yes,winter
4,2015-01-04 04:00:00,47,2.0,0.0,0.930,6.5,clear,No,yes,winter


In [14]:
#Export the dataframe 
df.to_excel('my_New_LBR.xlsx',sheet_name='LBR')
